<a href="https://colab.research.google.com/github/neal-logan/dsba6211-summer2024/blob/main/nophishing/01_exploratory_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# No Phishing: Detecting Malicious URLs

## Notebook 01: Exploratory Analysis


### Environment Setup

Developed with Python version 3.10.12 in Colab version 1.0.0

##### Install required packages


In [ ]:
#Docs: https://github.com/facebookresearch/hiplot
%%capture
!pip install hiplot==0.1.33

In [ ]:
#Docs: https://github.com/SelfExplainML/PiML-Toolbox
%%capture
!pip install PiML==0.6.0

##### Set random seed

In [ ]:
random_seed = 42

### Overview of the Data

The dataset consistes of the raw URL, the binary phishing label, and 87 features.

**Category of data source**:  Of the 87 features:
* 56 were created from URL syntax and structure,
* 24 were extracted from corresponding site content, and
* 7 were obtained from third-party services.

**Variable types**: The features are a mix of binary, integer, and floating-point variables.

**Distributions**: Variables tended to follow a few patters in their variation:
* Integer distributions: Some include only small numbers in the range of 1-10 and essentially ordinal categories, such as page_rank.  Others are counts of URL or site features that tend to get little larger than 50, and tend to be right-skewed.  Still others are strongly right-skewed and range into the hundreds or thousands, such as url_length or longest_word_raw.  
* Binary distributions: some are fairly well balanced while others show almost no variation at all.  
* Floating-point distributions: all floating point variables appear to be ratios between 0 and 1; some of them are distributed throughout that range, while others behave more like binary variables.

##### Load and Prepare Data

In [ ]:
# Load and prepare training data
import pandas as pd

train_url = 'https://raw.githubusercontent.com/neal-logan/dsba6211-summer2024/main/nophishing/data/phishing-url-pirochet-train.csv'
df = pd.read_csv(train_url)

#Create numeric target variable column
df['y'] = df['status'].replace('legitimate', 0).replace('phishing', 1)

#Drop unnecessary columns
df = df.drop(columns=['status','url'])

#X/y split
X = df.drop(columns=['y'])
y = df['y']

In [ ]:
#Split training set into training and validation set (test set not yet loaded)

from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(
    X,
    y,
    test_size = 0.2,
    random_state = random_seed)

In [ ]:
# Establish a list of columns being dropped from exploration and modeling
column_drop_list = ['url','status']

##### Categorize Features by Source
Based on data documentation

In [ ]:
# Split features into categories based on origin: from URL, from site content,
# or from third parties

url_columns = ['length_url', 'length_hostname', 'ip', 'nb_dots',
  'nb_hyphens', 'nb_at', 'nb_qm', 'nb_and', 'nb_or', 'nb_eq',
  'nb_underscore', 'nb_tilde', 'nb_percent', 'nb_slash', 'nb_star',
  'nb_colon', 'nb_comma', 'nb_semicolumn', 'nb_dollar', 'nb_space',
  'nb_www', 'nb_com', 'nb_dslash', 'http_in_path', 'https_token',
  'ratio_digits_url', 'ratio_digits_host', 'punycode', 'port',
  'tld_in_path', 'tld_in_subdomain', 'abnormal_subdomain',
  'nb_subdomains', 'prefix_suffix', 'random_domain', 'shortening_service',
  'path_extension', 'nb_redirection', 'nb_external_redirection',
  'length_words_raw', 'char_repeat', 'shortest_words_raw',
  'shortest_word_host', 'shortest_word_path', 'longest_words_raw',
  'longest_word_host', 'longest_word_path', 'avg_words_raw',
  'avg_word_host', 'avg_word_path', 'phish_hints', 'domain_in_brand',
  'brand_in_subdomain', 'brand_in_path', 'suspecious_tld',
  'statistical_report' ]

site_content_columns = ['nb_hyperlinks', 'ratio_intHyperlinks',
  'ratio_extHyperlinks', 'ratio_nullHyperlinks', 'nb_extCSS',
  'ratio_intRedirection', 'ratio_extRedirection', 'ratio_intErrors',
  'ratio_extErrors', 'login_form', 'external_favicon', 'links_in_tags',
  'submit_email', 'ratio_intMedia', 'ratio_extMedia', 'sfh', 'iframe',
  'popup_window', 'safe_anchor', 'onmouseover', 'right_clic',
  'empty_title', 'domain_in_title', 'domain_with_copyright']

third_party_columns = ['whois_registered_domain', 'domain_registration_length', 'domain_age',
       'web_traffic', 'dns_record', 'google_index', 'page_rank']


##### Summary statistics

In [ ]:
y_train.describe()

In [ ]:
X_train["page_rank"].describe()

print("Five-number Summary\nMin-25%-50%-75%-Max")
for col in X_train.columns:
  stats = X_train[col].describe()
  print(col + ": " + "{:.1f}".format(stats["min"]) + " - " + "{:.1f}".format(stats["25%"]) + " - " + "{:.1f}".format(stats["50%"]) + " - " + "{:.1f}".format(stats["75%"]) + " - " + "{:.1f}".format(stats["max"]))

### Invalid Feature Values

Just two features have apparently invalid values:
* domain_registration length has a handful of invalid observations affecting less than 1% of observations, while
* domain_age is invalid for 16% of observations; with the invalid values well-balanced between phishing and legitimate.

Sklearn's histogram gradient-boosted tree classifiers handle NaNs natively.   Since the main feature importance analysis and (more importantly) the main modeling efforts will use these classifiers, all negative values will be set to NaN.  However, they will not be removed until after baseline modeling, since the logistic regression and random forest classifier models can't handle NaNs.

In [ ]:
# For most features, values less than 0 will be invalid

for col in X_train.columns:
  invalid_count = X_train[X_train[col] < 0].shape[0]

  if(invalid_count > 0):
    X_train.shape[0]
    invalid_phishing = X_train[(X_train[col] < 0) & (y_train == 1)].shape[0]*1.0 / invalid_count
    print(col
          + ":\n   Invalid observations:  " + str(invalid_count)
          + "\n   Proportion of observations that are invalid:  " + str(invalid_count*1.0/X_train.shape[0])
          + "\n   Proportion of invalids that are phishing:  " + str(invalid_phishing))


### Feature-Target Correlation

Third-party features google_index and page_rank dominate the correlations, at about 0.74 and 0.50 respectively.  However, url features nb_www (corr=0.43) and ratio_digits_url (corr=0.36) also look promising, and they're closely followed by site content variables domain_in_title (corr=0.33) and nb_hyperlinks (corr=0.33).

On the other hand, most features show very little correlation, including several under 0.01.  And six features show no variation at all after removing the validation set from the original training set. (This was previously stable over a period of days at five, but is now stable at six. It's unclear why this changed, as the random seed and feature transformations have not changed).

Weakly-correlated features could still plausibly be useful to decision tree models, and will be retained at this stage.  

However, the following features do not vary in the training set can't be of any use, and will be immediately dropped from further analysis or modeling efforts:
* nb_or,
* ratio_nullHyperlinks,
* ratio_intRedirection,
* ratio_intErrors,
* submit_email,
* sfh


In [ ]:
# Recombine X,y training data sets for exploration
Xy_train = X_train.copy()
Xy_train['y'] = y_train

In [ ]:
# Calculate correlation matrix
corr_matrix = Xy_train.corr().abs()

# Get features most and least correlated with target variable
print(corr_matrix['y'].sort_values(ascending=False).head(45))
print(corr_matrix['y'].sort_values(ascending=False).tail(45))

In [ ]:
# Identify non-varying features
non_varying_droplist = []
for col in X_train.columns:
  if len(pd.unique(X_train[col])) < 2:
    non_varying_droplist.append(col)

print(non_varying_droplist)

##### Drop non-varying features

In [ ]:
# Add non-varying features to the drop list & drop from X_train

X_train.drop(columns=non_varying_droplist, inplace=True)
X_validation.drop(columns=non_varying_droplist, inplace=True)
Xy_train.drop(columns=non_varying_droplist, inplace=True)

### Initial parallel coordinate plots on correlated feature groups

I generated several groups of correlated features and plotted those groups (with the target variable) in parallel coordinate plots.

This exposed interesting relationships that couldn't be seen in the correlation matrix.  For example, tld_in_subdomain = 1 in only about 5% of observations, but the vast majority of these are phishing.  In a similar vein, ip = 1 accounts for about 15% of observations, of which about 5/6 are phishing.  Each of these could potentially serve as a red flag for phishing, even if they don't provide much information about most of the dataset.   

We can also see somewhat less narrowly-applicable features.  For example, setting a threshold of ratio_digits_url >= 0.05 gives us a little over 30% of observations, and of these almost 3/4 are phishing.  At nb_dots >= 4, we select about 13% of observations, nearly all phishing, but at nb_dots >= 3, this increases to about 33% of observations, which which almost 2/3 are phishing.

This pattern continues across numerous features: they may serve as red flags at higher thresholds, or may provide useful correlation at lower thresholds, or in some cases they may do both.

In addition to numerous features in these useful categories, a handful of features continued to show very strong potential. The google_index feature could form a phishing model by itself, and page_rank and domain_age each carry a lot of useful information.

##### Group correlated features

In [ ]:

import pandas as pd

#Create empty dataframes
dfs_eda = []
for i in range(0,13):
  df = pd.DataFrame()
  dfs_eda.append(df)

form_group_threshold = 0.3
join_group_threshold = 0.15


# Add the target variable to the first data frame to ensure
# highly-correlated variables will be explored
dfs_eda[0]['y'] = y_train

# Iterate through columns prioritizing those most correlated with the target
# grouping them based on correlation with columns already assigned to groups,
# correlation with the most-correlated other column, the presence of empty
# groups, and correlation thresholds.
for col in Xy_train:

  #Get the next most-correlated column other than col itself

  most_correlated = corr_matrix[col].sort_values(ascending=False).index[1]
  correlation = corr_matrix.loc[col, most_correlated]

  # Check if there are any remaining empty groups
  empty_remaining = False
  for df in dfs_eda:
    if df.empty:
      empty_remaining = True

  # If there are empty remaining groups and correlation for the current column
  # exceeds the threshold for group formation, add both to the first empty df
  if empty_remaining and correlation > form_group_threshold:
    for df in dfs_eda:
      if df.empty:
        df[col] = Xy_train[col]
        df[most_correlated] = Xy_train[most_correlated]
        break
  # Otherwise, if the correlation exceeds the minimum threshold for joining a
  # group
  elif corr_matrix['y'][col] > 0.3:
    dfs_eda[0][col] = Xy_train[col]
  elif correlation > join_group_threshold:
    for df in dfs_eda:
      if most_correlated in df.columns:
        df[col] = Xy_train[col]
        break
  elif corr_matrix['y'][col] > 0.15:
    dfs_eda[0][col] = Xy_train[col]


# Add the target variable to each group
for df in dfs_eda:
  if not 'y' in df.columns:
    df['y'] = y_train



##### Generate parallel coordinate plots

In [ ]:
import hiplot as hip

# convert dfs_eda to list of dicts because hiplot requires
dicts_eda = [df.to_dict('records') for df in dfs_eda]

for d in dicts_eda:
  hip.Experiment.from_iterable(d).display()

### Baseline Models

Simple logistic regression and random forest models are run at this stage to provide baseline models prior to assessing feature importance. Preprocessing consists only of sklearn's Standard Scaler at this stage.  Other scalers will be assessed later.

On the validation set, using all remaining features, two simple baseline models were run:
* **Logistic regression model**: **ROC-AUC > 0.94**
 * Iterations limit increased to 115 (default 100) to allow convergence
 * Overfitting concerns minimal
* **Random forest model**: **ROC-AUC > 0.95**
 * Estimators limited to 75 (default 100)
 * Tree depth limited to 7 (default none)
 * Both parameters selected to simplify model and avoid overtraining
 * Overfitting concerns minimal

##### Model performance evaluation function

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
import pandas as pd

# Define model evaluation function

def print_model_evaluation(
    title: str,
    pipe : Pipeline,
    X : pd.DataFrame,
    y : pd.DataFrame):

    print("\n" + title)
    pred_y = pipe.predict(X)

    print("\nROC-AUC: " + str(roc_auc_score(pred_y, y)))
    print("Precision: " + str(precision_score(pred_y, y)))
    print("Recall: " + str(recall_score(pred_y, y)))

##### **Logistic regression** model

In [ ]:
# Set up pipeline
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

pipe_lr = make_pipeline(
      StandardScaler(),
      LogisticRegression(
          random_state=random_seed,
          max_iter = 115 #Iterations increased to achieve convergence
          )
)

pipe_lr.fit(X_train, y_train)


In [ ]:
print_model_evaluation("Logistic Regression\nPerformance on Training Set",
                       pipe_lr, X_train, y_train)

print_model_evaluation("Logistic Regression\nPerformance on Validation Set",
                       pipe_lr, X_validation, y_validation)

##### **Random forest** model

In [ ]:
# Set up & run pipeline - random forest
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

from sklearn.preprocessing import QuantileTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline

pipe_rf = make_pipeline(
      QuantileTransformer(),
      RandomForestClassifier(
          n_estimators=75,
          max_depth=7,
          random_state=random_seed
          )
)

pipe_rf.fit(X_train, y_train)


In [ ]:
print_model_evaluation("Random Forest\nPerformance on Training Set",
                       pipe_rf, X_train, y_train)

print_model_evaluation("Random Forest\nPerformance on Validation Set",
                       pipe_rf, X_validation, y_validation)


### Feature Importance Analysis

**Preprocessing applied**: At this point, invalid values in domain_registration_length and domain_age were converted to NaN

**Model for feature importance analysis**: Initial assessments of feature importance for features selection were performed by generating a simple histogram-based gradient-boosted tree classifier.

**Permuation importance**: Sklearn's permutation importance was used to determine feature importance.

##### Convert invalid values

In [ ]:
#Convert invalid values to show they are missing
import numpy as np
X_train["domain_registration_length"] = [np.NaN if x < 0 else x for x in X_train["domain_registration_length"]]
X_validation["domain_registration_length"] = [np.NaN if x < 0 else x for x in X_validation["domain_registration_length"]]

X_train['domain_age'] = [np.NaN if x < 0 else x for x in X_train['domain_age']]
X_validation['domain_age'] = [np.NaN if x < 0 else x for x in X_validation['domain_age']]

##### Train histogram gradient-boosted tree model

In [ ]:
# Set up and run pipeline - gradient boosted trees
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingClassifier.html

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import make_pipeline

pipe_gbt = make_pipeline(
      StandardScaler(),
      HistGradientBoostingClassifier(
          max_iter = 60,
          max_depth = 7,
          random_state = random_seed)
)

pipe_gbt.fit(X_train, y_train)


In [ ]:
print_model_evaluation('Gradient-boosted Trees\nPerformance on Training Set',
                       pipe_gbt, X_train, y_train)

print_model_evaluation('Gradient-boosted Trees\nPerformance on Validation Set',
                       pipe_gbt, X_validation, y_validation)

##### Permutation importance

In [ ]:
from sklearn.inspection import permutation_importance

r = permutation_importance(pipe_gbt, X_validation, y_validation,
                           n_repeats=100,
                           random_state = random_seed)


In [ ]:
perm_imp = pd.DataFrame()
perm_imp['feature'] = X_validation.columns
perm_imp['importances_mean'] = r.importances_mean
perm_imp['importances_std'] = r.importances_std
perm_imp['importances_mean_less_std'] = r.importances_mean - r.importances_std
perm_imp = perm_imp.sort_values(by='importances_mean', ascending=False)

In [ ]:
perm_imp['importances_mean_less_std'].hist()

In [ ]:
perm_imp.head(20)

In [ ]:
perm_imp = perm_imp.sort_values(by='importances_mean_less_std', ascending=False)
perm_imp.head(25)

### Feature Selection for Main Modeling Effort

Feature importance was assessed using the validation set and sklearn's permutation importance function.  This analysis confirmed some previous suspicions, with only small surprises: google_index and page_rank dominated (expected) but very close together (somewhat unexpected). Just two other features (nb_www and nb_hyperlinks) showed mean importance >0.01, and a total of 19 showed mean importance >0.001.  

Feature importance varied widely.  Just 14 features show importance_mean-importance_std > 0 with importance_mean > 0.001. We can be reasonably confident that these features carry at least some useful information most of the time.

These 14 features will be used for the main modeling effort; all others will be rejected.

In [ ]:
selected_features = perm_imp[perm_imp['importances_mean_less_std'] > 0.0]
selected_features = selected_features[selected_features['importances_mean'] > 0.001]
print(selected_features['feature'])